In [1]:
import pandas as pd
import numpy as np
import pickle
pd.set_option('display.max_columns', None)
#pd.set_option('max_colwidth', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from IPython.display import Markdown, display
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

## Input Parameters

In [5]:
############### Input Parameters #####################

dat = pd.read_pickle('<data path>').round(4)           ### input data. Pandas DataFrame
numeric_percentiles = [.01,.25, .5, .75, .99]          ### percentiles for numeric variables
n = 3                                                  ### top n most frequent values of each categorical variable for display
display_num_var_count = 20                             ### can go upto the number of numeric variables available in data
display_cat_var_count = 10                             ### can go upto the number of categorical variables available in data
numeric_sort_by = 'count'
numeric_ascending = True
categorical_sort_by = 'unique'
categorical_ascending = False

#######################################################

## Variable Analysis

In [6]:
printmd("**Variable Distribution**", color = 'blue')
obs_count = dat.shape[0]
print("#obs in data:", obs_count)
coltypes = dat.dtypes.reset_index()
print('# of vars: ',coltypes.shape[0])
coltypes.columns = ['colname','datatype']
display(pd.DataFrame(coltypes.groupby('datatype').datatype.count()))
coltypes.loc[:,'coltype'] = 'numeric'
coltypes.loc[coltypes['datatype'] == 'object','coltype'] = 'categorical'
display(pd.DataFrame(coltypes.groupby(['coltype']).colname.count()))

cat_vars = coltypes.loc[coltypes['coltype'] == 'categorical','colname'].tolist()
numeric_vars = coltypes.loc[coltypes['coltype'] == 'numeric','colname'].tolist()

display(coltypes.head())

printmd("**Numeric Variable Analysis**", color = 'blue')
############################################################
numeric_vars = dat[numeric_vars].describe(percentiles = numeric_percentiles).transpose().drop('std', axis = 1)
#null_vars = numeric_vars.loc[numeric_vars['count']==0,:].reset_index()[['index']]['index'].tolist()
numeric_vars1 = numeric_vars.reset_index()
null_vars = numeric_vars1.loc[numeric_vars1['count']==0,'index'].tolist()
non_null_numeric = numeric_vars.loc[numeric_vars['count'] !=0,:]
non_null_numeric.insert(1, 'pct_nulls', 0)
non_null_numeric.loc[:,'pct_nulls'] = ((1-non_null_numeric.loc[:,'count']/obs_count)*100).round(2)
non_null_numeric.insert(0, 'obs_count', obs_count)

print("# numeric vars:",numeric_vars.shape[0])
print("# of numeric vars with all nulls:",len(null_vars))
print("# remaining vars:",non_null_numeric.shape[0])
print("Numeric vars with all nulls (first 10):",null_vars[:10])
printmd("***sorted by: " + numeric_sort_by + "***")
display(non_null_numeric.sort_values(numeric_sort_by, ascending = numeric_ascending).head(display_num_var_count))

printmd("**Categorical Variable Analysis**", color = 'blue')
############################################################
categorical_vars = dat[cat_vars].describe(include=[np.object]).transpose()
categorical_vars1 = categorical_vars.reset_index()
cat_null_vars = categorical_vars1.loc[categorical_vars1['count']==0,'index'].tolist()
#cat_null_vars = categorical_vars.loc[categorical_vars['count']==0,:].reset_index()[['index']]['index'].tolist()
print("# categorical vars:",categorical_vars.shape[0])
print("# of categorical vars with all nulls:",len(cat_null_vars))
non_null_cat = categorical_vars.loc[categorical_vars['count'] !=0,:]
non_null_cat.insert(1, 'pct_nulls', 0)
non_null_cat.loc[:,'pct_nulls'] = ((1-non_null_cat.loc[:,'count']/obs_count)*100)
print("# remaining vars:",non_null_cat.shape[0])
print("categorical vars with all nulls (first 10):",cat_null_vars[:10])

cols = list(range(n*2+1))

aggregates = pd.DataFrame(columns = cols)

for curr_var in cat_vars:
    #print("variable:", curr_var)
    top_categories = pd.DataFrame(dat[[curr_var]].groupby(curr_var)[curr_var].count().sort_values(0, ascending=False)).head(n).transpose()
    while top_categories.shape[1] < n:
        top_categories.loc[:,'_dummy_placeholder'+str(top_categories.shape[1])] = '_dummy_placeholder'+str(top_categories.shape[1])
    #display(top_categories,top_categories.shape[1])
    test = pd.DataFrame([curr_var] + top_categories.columns.tolist() + top_categories.iloc[0,:].tolist()).transpose()
    aggregates = pd.concat([aggregates,test],ignore_index=True)
    
colnames = ['index']
for k in range(n):
    currval = 'top_val'+str(k+1)
    colnames += [currval]
for k in range(n):
    freq = 'freq'+str(k+1)
    colnames += [freq]
for k in range(n):
    aggregates = aggregates.replace('_dummy_placeholder'+str(k), np.nan)
    
aggregates.columns = colnames
aggregates.iloc[:,-int(np.floor(len(cols)/2)):] = aggregates.iloc[:,-int(np.floor(len(cols)/2)):].fillna(-1).astype('int64')
#display(aggregates.head(), aggregates.shape)
top_categories = non_null_cat.reset_index().merge(aggregates, how = 'inner', \
                        on = ['index']).drop(['top','freq'],axis = 1).set_index('index')
top_categories.insert(0, 'obs_count', obs_count)

printmd("***sorted by: " + categorical_sort_by + "***")
display(top_categories.sort_values(categorical_sort_by, ascending = categorical_ascending).head(display_cat_var_count))

<span style='color:blue'>**Variable Distribution**</span>

#obs in data: 154463
# of vars:  717


,datatype
datatype,
float64,238
object,479


,colname
coltype,
categorical,479
numeric,238


,colname,datatype,coltype
0,var0,float64,numeric
1,var1,float64,numeric
2,var2,object,categorical
3,var3,object,categorical
4,var4,object,categorical


...

,obs_count,count,pct_nulls,mean,min,1%,25%,50%,75%,99%,max
var576,154463,16.00,99.99,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
var17,154463,16.00,99.99,49464.00,49464.00,49464.00,49464.00,49464.00,49464.00,49464.00,49464.00
var669,154463,31.00,99.98,187749.06,16594.00,16594.00,16594.00,333811.00,347732.00,361752.40,361795.00
var18,154463,32.00,99.98,2613.91,87.82,87.82,87.82,2613.91,5140.00,5140.00,5140.00
var20,154463,48.00,99.97,354596.33,78383.00,78383.00,78383.00,121406.00,864000.00,864000.00,864000.00
var19,154463,64.00,99.96,664.31,34.00,34.00,53.20,157.87,772.55,2308.00,2308.00
var578,154463,69.00,99.96,2017.58,2016.00,2016.00,2017.00,2018.00,2018.00,2019.00,2019.00
var654,154463,70.00,99.95,2012.77,2010.00,2010.00,2010.00,2011.00,2015.75,2018.00,2018.00
var653,154463,70.00,99.95,1035.67,0.06,0.06,0.10,4.00,317.00,12546.00,12546.00
var36,154463,114.00,99.93,7.78,0.00,0.00,0.00,2.14,13.06,26.05,26.05


<span style='color:blue'>**Categorical Variable Analysis**</span>

# categorical vars: 479
# of categorical vars with all nulls: 0
# remaining vars: 479
categorical vars with all nulls (first 10): []


<span style='color:None'>***sorted by: unique***</span>

,obs_count,count,pct_nulls,unique,top_val1,top_val2,top_val3,freq1,freq2,freq3
index,,,,,,,,,,
var348,154463,6756,95.63,646,0,10,12,509,114,107
var645,154463,24032,84.44,14,Practices - Governance and Strategy,Practices - Operations,Practices - Targets,18586,3871,604
var573,154463,7342,95.25,12,Sourcing Policy and Commitments - Percentage o...,Sourcing Policy and Commitments - Policy to ad...,Sourcing Policy and Commitments - Future targe...,3539,3169,329
var37,154463,10151,93.43,10,Policies & Disclosure - Clear policy on,Programs & Structures - Biodiversity and commu...,Programs & Structures,8101,406,356
var346,154463,13749,91.10,8,per million hours,"per 200,000 hours",per 100 employees,5543,4801,2024
var688,154463,778,99.50,7,On pace to achieve the ongoing target,Exceeded the set target,"Lack of data, status not determinable (ongoing...",286,153,144
var351,154463,16891,89.06,7,per million hours,"per 200,000 hours",per 100 employees,9092,4664,1776
var447,154463,20223,86.91,7,Performance - Energy Efficiency,Performance - Green Building,Performance - Sustainable Water,9582,3695,3595
var2,154463,14601,90.55,6,Data point/scoring missed,Data point incorrect,Incorrect application of guidelines,4681,4327,2418
